SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.

In [ ]:
import sqlite3

First, we need to create a new database and open a database connection to allow sqlite3 to work with it. Call sqlite3.connect() to create a connection to the database example.db in the current working directory, implicitly creating it if it does not exist:

In [3]:
## connect to sqllite3 database
connection = sqlite3.connect('example.db')

In [4]:
connection

The returned Connection object con represents the connection to the on-disk database.

In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor. Call con.cursor() to create the Cursor

In [ ]:
cur=connection.cursor()

Now that we’ve got a database connection and a cursor, we can create a database table movie with columns for title, release year, and review score. For simplicity, we can just use column names in the table declaration

In [6]:
cur.execute("CREATE TABLE Movie('tittle','year','score')")

We can verify that the new table has been created by querying the sqlite_master table built-in to SQLite, which should now contain an entry for the movie table definition (see The Schema Table for details). Execute that query by calling cur.execute(...), assign the result to res, and call res.fetchone() to fetch the resulting row:

In [7]:
res=cur.execute("SELECT Name FROM sqlite_master")
res.fetchone()

('Movie',)

Now, add two rows of data supplied as SQL literals by executing an INSERT statement, once again by calling cur.execute(...):



In [8]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

The INSERT statement implicitly opens a transaction, which needs to be committed before changes are saved in the database (see Transaction control for details). Call con.commit() on the connection object to commit the transaction:



In [11]:
connection.commit()

In [21]:
cur.execute("Select * from Movie")
rows=cur.fetchall()


In [23]:
for row in rows:
    print(row)

('Monty Python and the Holy Grail', 1975, 8.2)
('And Now for Something Completely Different', 1971, 7.5)
('Monty Python Live at the Hollywood Bowl', 1982, 7.9)
("Monty Python's The Meaning of Life", 1983, 7.5)
("Monty Python's Life of Brian", 1979, 8.0)
('Monty Python Live at the Hollywood Bowl', 1982, 7.9)
("Monty Python's The Meaning of Life", 1983, 7.5)
("Monty Python's Life of Brian", 1979, 8.0)


In [13]:
res = cur.execute("SELECT score FROM Movie")
res.fetchone()

(8.2,)

In [15]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
connection.commit()  # Remember to commit the transaction after executing INSERT.

In [17]:
for row in cur.execute("SELECT year, tittle FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")
(1983, "Monty Python's The Meaning of Life")


In [26]:
## update the table
cur.execute('''
 UPDATE Movie
 set score=10.0
 WHERE year=1971
''')


In [27]:
connection.commit()

In [29]:
cur.execute('SELECT * FROM Movie')
rows=cur.fetchall()

In [30]:
for row in rows:
    print(row)

('Monty Python and the Holy Grail', 1975, 8.2)
('And Now for Something Completely Different', 1971, 10.0)
('Monty Python Live at the Hollywood Bowl', 1982, 7.9)
("Monty Python's The Meaning of Life", 1983, 7.5)
("Monty Python's Life of Brian", 1979, 8.0)
('Monty Python Live at the Hollywood Bowl', 1982, 7.9)
("Monty Python's The Meaning of Life", 1983, 7.5)
("Monty Python's Life of Brian", 1979, 8.0)


In [32]:
## Deletion 
cur.execute('''
 Delete from Movie
 WHERE year=1982
''')

In [33]:
connection.commit()

In [34]:
for row in cur.execute("SELECT year FROM Movie"):
    print(row)

(1975,)
(1971,)
(1983,)
(1979,)
(1983,)
(1979,)


In [35]:
## close the connection
connection.close()

In [37]:
cur.execute('''
 Delete from Movie
 WHERE year=1982
''')

ProgrammingError: Cannot operate on a closed database.